In [59]:
import pandas as pd

# Data preprocessing

In [60]:
# Import imdb data
raw_folder = './raw_data/'
imdb_rating_folder = raw_folder + 'imdb_rating/'
cmu_folder = raw_folder + 'cmu/'
baby_names_folder = raw_folder + 'baby_names_national/'

rating_df = pd.read_csv(imdb_rating_folder + 'imdb_rating.tsv', sep='\t', index_col='tconst')
display(rating_df)

,averageRating,numVotes
tconst,,
tt0000001,5.7,2004
tt0000002,5.8,269
tt0000003,6.5,1904
tt0000004,5.5,178
tt0000005,6.2,2685
...,...,...
tt9916730,7.6,11
tt9916766,7.0,22
tt9916778,7.2,36


In [61]:
# Verify the indexes are unique
print(f"Is the indexing unique ? {rating_df.index.is_unique}")

Is the indexing unique ? True


# Importing Character MetaData

In [62]:
# Import character metadata
character_df = pd.read_csv(cmu_folder + 'character.metadata.tsv', sep='\t', header=None)

# Add column names deduced from README
character_df.columns = ['wiki_ID', 'free_ID', 'release', 'char_name', 'DOB', 'gender', 'height', 'ethnicity', 'act_name', 'age_at_release', 'free_char_map1', 'free_char_map2', 'free_char_map3']

display(character_df)

,wiki_ID,free_ID,release,char_name,DOB,gender,height,ethnicity,act_name,age_at_release,free_char_map1,free_char_map2,free_char_map3
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv
450665,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j
450666,28308153,/m/0cp05t9,1957,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44
450667,28308153,/m/0cp05t9,1957,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,NaN,/m/0g8ngmm


# Importing Movie MetaData

In [63]:
# Import movie metadata
movie_df = pd.read_csv(cmu_folder + 'movie.metadata.tsv', sep='\t', header=None)

# Add column names deduced from README
movie_df.columns = ['wiki_ID', 'free_ID', 'mov_name', 'release', 'revenue', 'runtime', 'languages', 'countries', 'genres']
display(movie_df)
print(f"Is the wiki_ID unique ? {movie_df.wiki_ID.is_unique}")

,wiki_ID,free_ID,mov_name,release,revenue,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


Is the wiki_ID unique ? True


In [64]:
movie_df.dtypes

wiki_ID        int64
free_ID       object
mov_name      object
release       object
revenue      float64
runtime      float64
languages     object
countries     object
genres        object
dtype: object

In [65]:
# # Get the index of the row with the smallest value in 'Column1'
# min_index = movie_df['release'].idxmin()

# # Get the entire row using the index
# min_row = movie_df.loc[min_index]
# print(f"oldest movies in CMU dataset : {min_row}")

### Question 1 

lets check the percentage of missing  values in the revenue attribut of the movie dataframe

In [66]:
nb_revenue_missing = movie_df['revenue'].isna().sum()
total_movies = len(movie_df)
perc_missing = (nb_revenue_missing / total_movies)*100
print(f"Percentage of missing values in column 'revenue': {perc_missing:.2f}%")

Percentage of missing values in column 'revenue': 89.72%


lets get a list of character name in the movie dataframe

The character dataframe has not a unique index. It is due to the several Nan values present in the same movie. To tackles this we can drop the rows that have a NaN as name and see if it solve the issue

In [67]:
character_filtered = character_df.copy(deep=True)

# drop the rows with a NaN as the character name
character_filtered = character_filtered.dropna(subset=['char_name'])
# drop the rows with the same character name
character_filtered = character_filtered.drop_duplicates(subset=['wiki_ID','char_name'])

In [68]:
character_filtered = character_filtered.set_index(['wiki_ID','char_name'])
display(character_filtered)

# Verify the indexes are unique
print(f"Is the indexing unique ? {character_filtered.index.is_unique}")

free_ID     release         DOB gender  \
wiki_ID char_name                                                              
975900  Akooshay                    /m/03vyhn  2001-08-24  1958-08-26      F   
        Lieutenant Melanie Ballard  /m/03vyhn  2001-08-24  1974-08-15      F   
        Desolation Williams         /m/03vyhn  2001-08-24  1969-06-15      M   
        Sgt Jericho Butler          /m/03vyhn  2001-08-24  1967-09-12      M   
        Bashira Kincaid             /m/03vyhn  2001-08-24  1977-09-25      F   
...                                       ...         ...         ...    ...   
913762  Maj. Nexx                   /m/03pcrp  1992-05-21  1960-04-28      M   
        Lord Feff                   /m/03pcrp  1992-05-21  1960-04-28      M   
        UN Spacy Commander          /m/03pcrp  1992-05-21        1954      M   
        Silvie Gena                 /m/03pcrp  1992-05-21        1958      F   
        Elensh                      /m/03pcrp  1992-05-21     1970-05      F   

                                    height   ethnicity            act_name  \
wiki_ID char_name                                                            
975900  Akooshay                     1.620         NaN      Wanda De Jesus   
        Lieutenant Melanie Ballard   1.780  /m/044038p  Natasha Henstridge   
        Desolation Williams          1.727     /m/0x67            Ice Cube   
        Sgt Jericho Butler           1.750         NaN       Jason Statham   
        Bashira Kincaid              1.650         NaN         Clea DuVall   
...                                    ...         ...                 ...   
913762  Maj. Nexx                      NaN         NaN         Steven Blum   
        Lord Feff                      NaN         NaN         Steven Blum   
        UN Spacy Commander             NaN         NaN       Sonny Byrkett   
        Silvie Gena                    NaN         NaN       Susan Byrkett   
        Elensh                         NaN         NaN  Dorothy Elias-Fahn   

                                    age_at_release free_char_map1  \
wiki_ID char_name                                                   
975900  Akooshay                              42.0     /m/0bgchxw   
        Lieutenant Melanie Ballard            27.0      /m/0jys3m   
        Desolation Williams                   32.0      /m/0jys3g   
        Sgt Jericho Butler                    33.0     /m/02vchl6   
        Bashira Kincaid                       23.0     /m/02vbb3r   
...                                            ...            ...   
913762  Maj. Nexx                             32.0     /m/0kr40d9   
        Lord Feff                             32.0     /m/0kr408g   
        UN Spacy Commander                    38.0     /m/0kr407w   
        Silvie Gena                           34.0     /m/0kr40b9   
        Elensh                                 NaN     /m/0kr406c   

                                   free_char_map2 free_char_map3  
wiki_ID char_name                                                 
975900  Akooshay                       /m/0bgcj3x     /m/03wcfv7  
        Lieutenant Melanie Ballard     /m/0bgchn4      /m/0346l4  
        Desolation Williams            /m/0bgchn_     /m/01vw26l  
        Sgt Jericho Butler             /m/0bgchnq      /m/034hyc  
        Bashira Kincaid                /m/0bgchp9      /m/01y9xg  
...                                           ...            ...  
913762  Maj. Nexx                      /m/0kr40df      /m/044_7j  
        Lord Feff                      /m/0kr408l      /m/044_7j  
        UN Spacy Commander             /m/0kr407_      /m/0gn4bz  
        Silvie Gena                    /m/0kr40bf      /m/0gn4nd  
        Elensh                         /m/0kr406h      /m/0b_vcv  

[189341 rows x 11 columns]

Is the indexing unique ? True


In [69]:
name_df = character_filtered.reset_index()[['wiki_ID', 'char_name']].copy(deep=True)
display(name_df)

,wiki_ID,char_name
0,975900,Akooshay
1,975900,Lieutenant Melanie Ballard
2,975900,Desolation Williams
3,975900,Sgt Jericho Butler
4,975900,Bashira Kincaid
...,...,...
189336,913762,Maj. Nexx
189337,913762,Lord Feff
189338,913762,UN Spacy Commander
189339,913762,Silvie Gena


In [70]:
# Split the character names into words and explode the lists
exploded_name_df = name_df.assign(char_words=name_df['char_name'].str.split()).explode('char_words')
word_name_df = exploded_name_df.drop(columns=['char_name'])
print("exploded_name_df :")
display(exploded_name_df)
print("word_name_df :")
display(word_name_df)

exploded_name_df :


,wiki_ID,char_name,char_words
0,975900,Akooshay,Akooshay
1,975900,Lieutenant Melanie Ballard,Lieutenant
1,975900,Lieutenant Melanie Ballard,Melanie
1,975900,Lieutenant Melanie Ballard,Ballard
2,975900,Desolation Williams,Desolation
...,...,...,...
189338,913762,UN Spacy Commander,Spacy
189338,913762,UN Spacy Commander,Commander
189339,913762,Silvie Gena,Silvie
189339,913762,Silvie Gena,Gena


word_name_df :


,wiki_ID,char_words
0,975900,Akooshay
1,975900,Lieutenant
1,975900,Melanie
1,975900,Ballard
2,975900,Desolation
...,...,...
189338,913762,Spacy
189338,913762,Commander
189339,913762,Silvie
189339,913762,Gena


In [71]:
# # Your original DataFrame
# data = {
#     'movie_id': [1, 1, 2, 2, 3],
#     'character_name': ['Alice', 'Bob', 'Charlie David', 'Eve', 'Frank Grace']
# }

# df = pd.DataFrame(data)

# # Split the character names into words and explode the lists
# exploded_df = df.assign(character_words=df['character_name'].str.split()).explode('character_words')

# # Reset the index for a cleaner result
# exploded_df.reset_index(drop=True, inplace=True)

# # Print the original and the new DataFrame
# print("Original DataFrame:")
# display(df)

# print("\nExploded DataFrame:")
# display(exploded_df)

In [72]:
# # DataFrame with only one entry containing all character names for a given film
# character_per_movie_df = character_filtered.groupby('wiki_ID').apply(lambda x: pd.Series({
#         'char_name': x['char_name'].values
#     }))
# display(character_per_movie_df)

In [73]:
# df_movie_names = character_per_movie_df[0:]

# def split_names(names_list):
#     result = []
#     for name in names_list:
#         split_name = name.split()
#         result.extend(split_name)
#     return result

# df_movie_names['split_names'] = df_movie_names['char_name'].apply(split_names)

# display(df_movie_names)

### Import given name dataset

In [74]:
# import name dataset
import os

# Define the path to your dataset folder
folder_path = 'raw_data/baby_names_national/'

# Create an empty list to store individual DataFrames
data_frames = []

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.startswith('yob') and filename.endswith('.txt'):
        # Extract the year from the filename
        year = int(filename[3:-4])

        # Read the data from the file into a DataFrame
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, header=None, names=['name', 'gender', 'number'])

        # Add the 'year' column to the DataFrame
        df['year'] = year

        # Append the current DataFrame to the list
        data_frames.append(df)

# Concatenate all DataFrames in the list into one DataFrame
baby_name_df = pd.concat(data_frames, ignore_index=True)

# Set the 'name' column as the index
# combined_data.set_index('name', inplace=True)

# Display the resulting DataFrame
baby_name_df.head()

,name,gender,number,year
0,Mary,F,7065,1880
1,Anna,F,2604,1880
2,Emma,F,2003,1880
3,Elizabeth,F,1939,1880
4,Minnie,F,1746,1880


### Filter the names to keep only the ones available in the baby name dataset

In [75]:
# does the word_name_df has duplicates?
duplicates = word_name_df.duplicated()
print(f"number of duplicates in word_name_df = {duplicates.sum()}")
word_name_df = word_name_df.drop_duplicates()
duplicates_after = word_name_df.duplicated()
print(f"number of duplicates in word_name_df = {duplicates_after.sum()}")

number of duplicates in word_name_df = 28595
number of duplicates in word_name_df = 0


In [76]:
baby_name_only_df = baby_name_df[['name']].copy(deep=True)

# does the baby_name_only_df has duplicates?
duplicates = baby_name_only_df.duplicated()
print(f"number of duplicates in baby_name_only_df = {duplicates.sum()}")
baby_name_only_df = baby_name_only_df.drop_duplicates()
duplicates = baby_name_only_df.duplicated()
print(f"number of duplicates in baby_name_only_df = {duplicates.sum()}")

number of duplicates in baby_name_only_df = 1982709
number of duplicates in baby_name_only_df = 0


In [77]:
print("word_name_df :")
print(f"   type : {type(word_name_df)}")
display(word_name_df)

print("baby_name_only_df :")
print(f"   type : {type(baby_name_only_df)}")
display(baby_name_only_df)

# Use pd.merge to filter rows based on 'names' column
word_name_filtered_df = pd.merge(word_name_df.reset_index(), baby_name_only_df, left_on='char_words', right_on='name', how='inner')
# word_name_filtered_df = word_name_filtered_df.drop_duplicates()
word_name_filtered_df.set_index(['wiki_ID', 'char_words'], inplace=True)
print("word_name_filtered_df :")
display(word_name_filtered_df)

# Verify the indexes are unique
print(f"Is the indexing unique ? {word_name_filtered_df.index.is_unique}")

word_name_df :
   type : <class 'pandas.core.frame.DataFrame'>


,wiki_ID,char_words
0,975900,Akooshay
1,975900,Lieutenant
1,975900,Melanie
1,975900,Ballard
2,975900,Desolation
...,...,...
189338,913762,Spacy
189338,913762,Commander
189339,913762,Silvie
189339,913762,Gena


baby_name_only_df :
   type : <class 'pandas.core.frame.DataFrame'>


,name
0,Mary
1,Anna
2,Emma
3,Elizabeth
4,Minnie
...,...
2085144,Zillion
2085145,Ziri
2085147,Ziyaire
2085152,Zoumana


word_name_filtered_df :


,,index,name
wiki_ID,char_words,,
975900,Lieutenant,1,Lieutenant
7668793,Lieutenant,1278,Lieutenant
24226493,Lieutenant,2314,Lieutenant
3388805,Lieutenant,3228,Lieutenant
8231713,Lieutenant,3611,Lieutenant
...,...,...,...
24341999,Dagen,189091,Dagen
43452,Auda,189118,Auda
11372598,Bonni,189134,Bonni


Is the indexing unique ? True


In [78]:
# check for the first movie of the CMU dataset, seems ok
word_name_filtered_df.loc[975900]

,index,name
char_words,,
Lieutenant,1,Lieutenant
Melanie,1,Melanie
Ballard,1,Ballard
Williams,2,Williams
Jericho,3,Jericho
Butler,3,Butler
Bashira,4,Bashira
Kincaid,4,Kincaid
Helena,5,Helena


In [79]:
# is Lieutenant in the baby name dataset?
name_to_search = 'Lieutenant'

# Check if the name is present
is_name_present = name_to_search in baby_name_only_df['name'].values

if is_name_present:
    print(f"{name_to_search} is present in the DataFrame.")
else:
    print(f"{name_to_search} is not present in the DataFrame.")

Lieutenant is present in the DataFrame.


In [80]:
# # Your data
# data_A = {
#     'movie': [1, 1, 1, 2, 2, 3, 3, 4, 4],
#     'names_a': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Alice', 'Frank', 'Grace', 'Hank'],
#     'number': [12, 13, 14, 15, 16, 17, 18, 19, 20]
#     }
# df_A = pd.DataFrame(data_A)
# # df_A.set_index(['movie', 'names_a'], inplace=True)

# data_B = {'names_b': ['Alice', 'Bob', 'Eve', 'Hank', 'Maxime']}
# df_B = pd.DataFrame(data_B)

# # Use pd.merge to filter rows based on 'names' column
# df_A_filtered = pd.merge(df_A, df_B, left_on='names_a', right_on='names_b', how='inner')

# print("Original df_A:")
# print(f"   type : {type(df_A)}")
# display(df_A)

# print("\ndf_B:")
# print(f"   type : {type(df_B)}")
# display(df_B)

# print("\nFiltered df_A:")
# print(f"   type : {type(df_A_filtered)}")
# display(df_A_filtered)

# Thomas 

In [81]:
#Character names analysis

df_movie_names.head()

df_movie_names['split_names']

print(type(df_movie_names['split_names'].iloc[0]))



NameError: name 'df_movie_names' is not defined

In [ ]:
# Faire une series des nom unique des enfants 
unique_entries = combined_data['name'].unique()

# Convert the unique entries to a list
unique_entries_list = list(unique_entries)

#   display(unique_entries_list)

print(df_movie_names['split_names'].iloc[0])

intersection_set = set(df_movie_names['split_names'].iloc[0]) & set(unique_entries_list)
print(intersection_set)

bool(intersection_set)

In [ ]:
len(df_movie_names.split_names)

Intersection_names = []

#for i in range(10) : 

#    inter = set(df_movie_names['split_names'].iloc[i]) & set(unique_entries_list)

#    Intersection_names = [Intersection_names,bool(inter)]

Intersection_names = []

for i in range(100):
    inter = set(df_movie_names['split_names'].iloc[i]) & set(unique_entries_list)
    Intersection_names.append(bool(inter))

Intersection_names

In [ ]:
#display(Intersection_names)

# To keep in mind

Idée pour ne pas utiliser des listes pour les noms dans une dataframe

In [ ]:
# Sample data
movies_data = {'MovieTitle': ['Movie1', 'Movie2'],
               'CharacterNames': [['Character1', 'Character2'], ['Character3', 'Character4', 'Character5']]}

# Creating a DataFrame
test_df = pd.DataFrame(movies_data)

# Convert the list of character names to a string, separated by a delimiter (e.g., ';')
test_df['CharacterNames'] = test_df['CharacterNames'].apply(lambda x: ';'.join(x))
display(test_df)

,MovieTitle,CharacterNames
0,Movie1,Character1;Character2
1,Movie2,Character3;Character4;Character5


Ensuite pour retrouver une liste de noms, on peut faire

In [82]:
# Split the string back into a list
test_df['CharacterNamesList'] = test_df['CharacterNames'].apply(lambda x: x.split(';'))
display(test_df)

,MovieTitle,CharacterNames,CharacterNamesList
0,Movie1,Character1;Character2,"[Character1, Character2]"
1,Movie2,Character3;Character4;Character5,"[Character3, Character4, Character5]"
